In [71]:
# Importing modules

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

In [72]:
# Reading training data and dropping time column

pd.set_option('display.max_columns', None)
df = pd.read_csv('dataset/train/training-data.csv')
df = df.drop(['time'], axis=1)
df

,f1_c,f1_a,f1_s,f1_d,f2_c,f2_a,f2_s,f2_d,prg_c,prg_a,prg_s,prg_d,prd_c,prd_a,prd_s,prd_d,pr_s,pr_d,lq_s,lq_d,cmp_a_s,cmp_a_d,cmp_b_s,cmp_b_d,cmp_c_s,cmp_c_d
0,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
1,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
2,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
3,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
4,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9780,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0
9781,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0
9782,42919,53824,65535,0,1868,1895,161,-2,0,0,0,0,52984,51424,21841,692,55163,60,28991,-4,30828,0,980,0,33725,0
9783,42919,53824,65535,0,1868,1895,161,0,0,0,0,0,52984,51424,21841,0,55163,0,28991,0,30828,0,980,0,33725,0


In [73]:
# Function that computes mean and variance for features

def compute_mu_and_variance(df):
    mu_arr = []
    variance_arr = []
    for column in df:
        arr = df[column].to_numpy()
        mu = np.average(arr)
        std = np.sqrt(np.mean((arr - np.mean(arr)) ** 2))
        variance = np.mean((arr - np.mean(arr)) ** 2)
        mu_arr.append(mu)
        variance_arr.append(variance)
    return mu_arr, variance_arr

In [74]:
# Applying guassian model and dropping unimportant features

def guassian_proabability(df, mu_arr, variance_arr):
    count = 0
    sub_df = df.iloc[:, :27]
    for col in sub_df:
        prob_list = []
        arr = sub_df[col].to_numpy()
        for i in arr:
            prob = (1/(np.sqrt(variance_arr[count]) * np.sqrt(2*np.pi)))*np.e**(-((i - mu_arr[count])**2)/(2*variance_arr[count]))
            prob_list.append(prob)
        df[list(df.columns)[count] + '_prob'] = prob_list
        count += 1
    df = df.drop(columns=['f1_c_prob', 'f1_a_prob', 'f1_s_prob', 'f1_d_prob', 'f2_c_prob', 'f2_a_prob', 'f2_s_prob', 'prg_a_prob', 'prg_c_prob', 'prg_s_prob', 'prd_c_prob', 'prd_a_prob', 'prd_s_prob', 'pr_s_prob', 'lq_s_prob', 'cmp_a_s_prob', 'cmp_b_s_prob', 'cmp_c_s_prob'])
    sub_prod_df = df.iloc[:,26:35]
    df['p_of_x'] = sub_prod_df['f2_d_prob'] * sub_prod_df['prg_d_prob'] * sub_prod_df['prd_d_prob'] * sub_prod_df['pr_d_prob'] * sub_prod_df['lq_d_prob'] * sub_prod_df['cmp_a_d_prob'] * sub_prod_df['cmp_b_d_prob'] * sub_prod_df['cmp_c_d_prob']
    return df

In [75]:
# Getting mean, variance from features and applying the model to get probabilities

mu_arr, variance_arr = compute_mu_and_variance(df)
df = guassian_proabability(df, mu_arr, variance_arr)
df

,f1_c,f1_a,f1_s,f1_d,f2_c,f2_a,f2_s,f2_d,prg_c,prg_a,prg_s,prg_d,prd_c,prd_a,prd_s,prd_d,pr_s,pr_d,lq_s,lq_d,cmp_a_s,cmp_a_d,cmp_b_s,cmp_b_d,cmp_c_s,cmp_c_d,f2_d_prob,prg_d_prob,prd_d_prob,pr_d_prob,lq_d_prob,cmp_a_d_prob,cmp_b_d_prob,cmp_c_d_prob,p_of_x
0,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
1,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
2,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
3,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
4,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9780,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
9781,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09
9782,42919,53824,65535,0,1868,1895,161,-2,0,0,0,0,52984,51424,21841,692,55163,60,28991,-4,30828,0,980,0,33725,0,0.000126,0.00007,1.947107e-07,0.001216,0.018768,1.278333,1.508892,2.762855,2.096817e-19
9783,42919,53824,65535,0,1868,1895,161,0,0,0,0,0,52984,51424,21841,0,55163,0,28991,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09


In [76]:
# Preparing cross validation data for tuning

cv_dir = 'dataset/valid'
valid_filenames = glob.glob(cv_dir + "\*.csv")

for i in valid_filenames:
    cv_df = pd.read_csv(i)
    cv_df = cv_df.drop(['time'], axis=1)
    print(cv_df.head(3))
    

    f1_c   f1_a   f1_s  f1_d  f2_c  f2_a  f2_s  f2_d  prg_c  prg_a  prg_s  \
0  65535  65535  65535     0  2519  2506   212     0      0      0      0   
1  65535  65535  65535     0  2519  2506   212     0      0      0      0   
2  65535  65535  65535     0  2519  2506   212     0      0      0      0   

   prg_d  prd_c  prd_a  prd_s  prd_d   pr_s  pr_d   lq_s  lq_d  cmp_a_s  \
0      0   5641   6731   2840      0  54480     0  28726     0    30788   
1      0   5641   6731   2840      0  54480     0  28726     0    30788   
2      0   5641   6731   2840      0  54480     0  28726     0    30788   

   cmp_a_d  cmp_b_s  cmp_b_d  cmp_c_s  cmp_c_d  
0        0     1173        0    33572        0  
1        0     1173        0    33572        0  
2        0     1173        0    33572        0  
    f1_c   f1_a   f1_s  f1_d  f2_c  f2_a  f2_s  f2_d  prg_c  prg_a  prg_s  \
0  65535  65535  57343     0  2519  2506   212     0      0      0      0   
1  65535  65535  57343     0  2519  2506